In [ ]:
# default_exp model

In [ ]:
#export
from fastai.tabular.all import *
from fgcnn.data import *

In [ ]:
#hide
from nbdev.showdoc import *

# Model Architecture

> Module to create model architecture

The methods provide way to create train and test datasets, splitter and dataloaders.

### Prepare dataloaders

In [ ]:
#slow
dls = get_dl()
dls.show()

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,click_hour,C14,C15,C16,C17,C18,C19,C20,C21,click
3280536,1010,1,85f751fd,c4e18dd6,50e219e0,826c565e,7801e8d9,0f2161f8,8a7fb045,be23dbfa,f07e20f8,4,0,16,21790,320,50,2513,3,35,-1,68,0
10013407,1002,0,1ee69d19,ce81106c,50e219e0,ecad2386,7801e8d9,07d7df22,60b5cd1b,e2c233db,507de649,0,0,04,15908,320,50,1752,3,297,100081,82,1
7790564,1005,0,85f751fd,c4e18dd6,50e219e0,36f58dec,2347f47a,f95efa07,a99f214a,8e787681,d780319b,1,0,13,21767,320,50,2506,0,35,-1,157,0
6718210,1005,0,85f751fd,c4e18dd6,50e219e0,e9739828,df32afa9,cef3e649,a99f214a,d6c0acb1,e55b4de4,1,0,10,21768,320,50,2506,0,35,100020,157,0
4512824,1005,0,85f751fd,c4e18dd6,50e219e0,66f5e02e,6f7ca2ba,0f2161f8,a99f214a,fc895806,2891f384,1,0,03,21678,320,50,2495,2,167,-1,23,0
6886687,1002,0,85f751fd,c4e18dd6,50e219e0,a37bf1e4,7801e8d9,07d7df22,dce843f8,97d296f6,3bcda2fe,0,0,11,14265,320,50,1526,2,169,100111,35,0
6909109,1005,0,cd58172f,b9c4ab81,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,cfcbdff7,158e4944,1,0,11,17877,320,50,2036,3,47,-1,156,0
648145,1005,0,23d99ea0,6bdbd889,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,339649fc,6e1e2240,1,0,03,16208,320,50,1800,3,167,100074,23,0
5713179,1005,0,4e7614cf,c1aa3c04,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e70f5b2e,8ef046ab,1,0,08,21764,216,36,2506,0,35,100076,157,0
3903661,1005,0,85f751fd,c4e18dd6,50e219e0,7358e05e,b9528b13,cef3e649,8c2d936a,4b44384d,be6db1d7,1,0,21,456,320,50,122,3,1319,-1,15,0


### Embeddings

In [ ]:
#export
def _fixed_emb_sz(classes, n, k):
    "Pick a fixed embedding size for `n` based on `k`."
    n_cat = len(classes[n])
    sz    = k  # rule of thumb
    return n_cat,sz

def get_emb_sz(to, k):
    "Get default embedding size from `TabularPreprocessor` `proc` or the ones in `sz_dict`"
    return [_fixed_emb_sz(to.classes, n, k) for n in to.cat_names]

In [ ]:
#slow
get_emb_sz(dls, k=40)

[(8, 40),
 (8, 40),
 (3513, 40),
 (4608, 40),
 (24, 40),
 (5501, 40),
 (392, 40),
 (34, 40),
 (787082, 40),
 (2157569, 40),
 (6885, 40),
 (5, 40),
 (5, 40),
 (25, 40),
 (1089, 40),
 (9, 40),
 (10, 40),
 (239, 40),
 (5, 40),
 (51, 40),
 (169, 40),
 (44, 40)]

## FGCNN

Convolution -> Max Pooling -> Recombination

In [ ]:
#export
class ConvPoolRecombine(Module):
    """
    input : ( N, C_in, H, W ), where w represents `k` ( emebdding size )
    output: ( N, new_i, H / 2, W), where `2` represents max pool kernel size which is fixed as `2` for now.
    """
    
    def __init__(self, ch_in, ch_out, recomb_ch_out, out_wh, h, hp):
        store_attr()
        
        self.conv = nn.Conv2d(in_channels=ch_in,
                              out_channels=ch_out,
                              kernel_size=(h, 1),
                              padding='same',
                              stride=(1, 1)
                             )
        
        self.tanh   = nn.Tanh()
        self.pool   = nn.MaxPool2d(kernel_size=(hp, 1))
        
        self.recomb = nn.Linear(out_wh*ch_out, out_wh*recomb_ch_out)
    
    def forward(self, x):
        batch_size = x.shape[0]
        embed_size = x.shape[3]
        
        # output shape: (N, C_out, H, W)
        c = self.conv(x)
        c = self.tanh(c)
        
        # output shape: (N, C_out, H / hp, W), where hp = `2`
        p = self.pool(c)
        
        # output shape: (N, H / hp, W, C_out)
        f = p.permute([0, 2, 3, 1]).contiguous()
        
        # flattening
        # shape: (N, H/2*W*C_out)
        f = f.view(batch_size, -1)
        
        # recombining
        # shape: (N, H/2, W, C_new)
        r = self.recomb(f).view(batch_size, -1, embed_size, self.recomb_ch_out)
        
        # shape: (N, c_new*H/hp, W)
        out_r = r.permute([0, 3, 1, 2]).contiguous().view(batch_size, -1, embed_size)
        
        return p, out_r

In [ ]:
#export
class FGCNN(Module):
    def __init__(self, emb_szs, conv_kernels, kernels, dense_layers, h, hp):
        """
        emb_szs     : list of tuples representing embedding size e.g. [(4, k), (6, k)]
        conv_kernels: list of convolutional kernels
        kernels     : kernels to be used
        h           : convolutional filter size
        hp          : pooling kernel size
        """
        
        self.k        = emb_szs[0][1]
        self.n_fields = len(emb_szs) 
        self.embeds   = nn.ModuleList([Embedding(ni, nf) for ni,nf in emb_szs])
        self.n_emb    = sum(e.embedding_dim for e in self.embeds)
        
        self.conv_layers = nn.ModuleList([ConvPoolRecombine(ch_in=1 if i == 0 else conv_kernels[i-1], 
                                                            ch_out=conv_kernels[i], 
                                                            recomb_ch_out=kernels[i],
                                                            out_wh=int(self.n_fields / (2 ** (i + 1))) * self.k,
                                                            h=h, 
                                                            hp=hp
                                                           ) for i in range(len(conv_kernels))
                                         ])
        
        N = np.sum([int(self.n_fields / (hp ** (i + 1))) for i in range(len(conv_kernels))]) * kernels[0]
        
        self.lin_in = self.n_fields * self.k + int((N + self.n_fields) * (N + self.n_fields - 1) / 2)
        
        # MLP classifier
        self.act = nn.ReLU()
        self.mlp = nn.ModuleList([LinBnDrop(n_in=self.lin_in if i == 0  else dense_layers[i-1], 
                                            n_out=dense_layers[i],
                                            act=self.act
                                            ) 
                                          for i in range(len(dense_layers))
                                 ])
         
        self.final_linear = nn.Linear(in_features=dense_layers[-1], out_features=1, bias=False)
        self.sigmoid      = nn.Sigmoid()
        
        store_attr()
    
    def forward(self, x, x_cont):
        bs = x.shape[0]
        
        if self.n_emb != 0:
            x = [e(x[:,i]).unsqueeze(dim=1) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            
        embed = x.clone()
        
        input_x = x.view(bs, 1, self.n_fields, self.k)
        
        p     = input_x
        out_r = []
        
        for i in range(len(self.conv_layers)):
            p, r = self.conv_layers[i](p)
            out_r.append(r)
        
        # shape: (bs, C_new * ( H / 2 ^ i), embed_size)
        new_features = torch.cat(out_r, dim=1)
        
        # combine features
        aug_emb = torch.cat([embed, new_features], dim=1)
        
        # inner product factorization machine using augmented embedding matrix
        fm = []
        
        for i in range(aug_emb.shape[1]):
            for j in range(i+1, aug_emb.shape[1]):
                fm.append(torch.sum(torch.mm(aug_emb[:,i,:],
                                             aug_emb[:,j,:].T), dim=1
                                   ).unsqueeze(dim=1))

        fm  = torch.cat(fm, dim=1)
        
        # flatten
        out = torch.cat([fm, embed.view(bs, -1)], dim=1)
        
        for i in range(len(self.mlp)):
            out = self.mlp[i](out)
            
        out = self.final_linear(out)
        out = self.sigmoid(out)
        
        return out